<a href="https://colab.research.google.com/github/KozinIS/CV/blob/main/13_%D0%A7%D0%B0%D1%82_%D0%B1%D0%BE%D1%82_%D0%BF%D0%BE_%D0%B7%D0%B0%D1%89%D0%B8%D1%82%D0%B5_%D0%B8%D0%BD%D1%84%D0%BE%D1%80%D0%BC%D0%B0%D1%86%D0%B8%D0%B8_%D0%B2_%D0%BA%D1%80%D0%B8%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B9_%D0%B8%D0%BD%D1%84%D0%BE%D1%80%D0%BC%D0%B0%D1%86%D0%B8%D0%BE%D0%BD%D0%BD%D0%BE%D0%B9_%D0%B8%D0%BD%D1%84%D1%80%D0%B0%D1%81%D1%82%D1%80%D1%83%D0%BA%D1%82%D1%83%D1%80%D0%B5_(ChatGPT_%2B_API).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Чат-бот по защите информации в критической информационной инфраструктуре (ChatGPT + API)**

# 1 Постановка задачи

*Возьмите в интернете любую инструкцию по эксплуатации и на ее основе сделайте базу знаний для ответа на вопросы по этой инструкции. Для этого разбейте взятую инструкцию на логические блоки (как в занятии), и уже эти логические блоки разбейте сплиттерами на чанки. Создайте нейро-консультанта по данной базе знаний - с поддержанием диалога или без такового - на ваше усмотрение. Проверьте работу созданного нейро-консультанта.*

# 2 Подгрузка библиотек

In [ ]:
!pip  install  tiktoken  langchain openai chromadb gspread oauth2client nltk pydantic==1.10.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 51.3 MB/s et

In [ ]:
#@title Импорт библиотек
import gdown
from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re
import getpass
import os
import openai
import tiktoken

# 3 Подключение к API

In [ ]:
# Получение ключа API от пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


# 4 Подготовка функций

In [ ]:
def split_text(text, max_count, count_type, verbose=0):
    # Функция для подсчета количества слов в фрагменте
    def num_words(fragment):
        return len(fragment.split())

    # Функция для подсчета количества токенов в фрагменте
    def num_tokens(fragment):
        return num_tokens_from_string(fragment, "cl100k_base")

    # Разделение текста на фрагменты, исключая теги HTML
    fragments = [fragment.strip() for fragment in re.split(r"<[^>]+>|[\ufeff]", text) if fragment.strip()]

    # Выбор функции подсчета длины в зависимости от типа подсчета
    length_function = num_words if count_type == "words" else num_tokens

    # Создание объекта разделителя текста
    splitter = RecursiveCharacterTextSplitter(chunk_size=max_count, chunk_overlap=0, length_function=length_function)

    # Список для хранения фрагментов текста
    source_chunks = []

    # Обработка каждого фрагмента текста
    for fragment in fragments:
        if verbose:
            # Вывод количества слов/токенов в фрагменте, если включен режим verbose
            count = length_function(fragment)
            print(f"{count_type} in text fragment = {count}\n{'-' * 5}\n{fragment}\n{'=' * 20}")

        # Разбиение фрагмента текста на части заданной длины с помощью разделителя
        # и добавление каждой части в список source_chunks
        source_chunks.extend(Document(page_content=chunk, metadata={}) for chunk in splitter.split_text(fragment))

    # Возвращение списка фрагментов текста
    return source_chunks


def create_embedding(data, max_count, count_type):
    def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Возвращает количество токенов в строке"""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

    source_chunks = []

    source_chunks = split_text(text=data, max_count=max_count, count_type=count_type, verbose=0)

    # Создание индексов документа
    search_index = Chroma.from_documents(source_chunks, OpenAIEmbeddings(), )

    count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
    print('\n ===========================================: ')
    print('Количество токенов в документе :', count_token)
    print('ЦЕНА запроса:', 0.0004*(count_token/1000), ' $')
    return search_index

def load_search_indexes(url: str, max_count, count_type) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text
    return create_embedding(text, max_count=max_count, count_type=count_type)

In [ ]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
        num_tokens = 0
        for message in messages:
            num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":  # if there's a name, the role is omitted
                    num_tokens += -1  # role is always required and always 1 token
        num_tokens += 2  # every reply is primed with <im_start>assistant
        return num_tokens
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.""")

def insert_newlines(text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

def answer_index(system, topic, search_index, temp = 1, verbose = 0, top_similar_documents = 5):

    #Выборка документов по схожести с вопросом
    docs = search_index.similarity_search(topic, k=top_similar_documents)
    if (verbose): print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if (verbose): print('message_content :\n ======================================== \n', message_content)

    messages = [
      {"role": "system", "content": system + f"{message_content}"},
      {"role": "user", "content": topic}
      ]

    # example token count from the function defined above
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f"{num_tokens_from_messages(messages, 'gpt-3.5-turbo-0301')} токенов использовано на вопрос")

    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=temp
    )
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f'{completion["usage"]["total_tokens"]} токенов использовано всего (вопрос-ответ).')
    if (verbose): print('\n ===========================================: ')
    if (verbose): print('ЦЕНА запроса с ответом :', 0.002*(completion["usage"]["total_tokens"]/1000), ' $')
    if (verbose): print('\n ===========================================: ')
    print('ОТВЕТ : \n', insert_newlines(completion.choices[0].message.content))

    # return completion

# 5 Задание параметров

In [ ]:
max_count = 110
count_type = "words"
top_similar_documents = 5

In [ ]:
# Документ "Приказ ФСТЭК России от 25.12.2017 № 239
# "Об утверждении Требований по обеспечению безопасности значимых объектов критической информационной инфраструктуры Российской Федерации"
manual_index = load_search_indexes('https://docs.google.com/document/d/1KLCWyX0XJYvuaDFUdNfBeG2-NhiaI0t3/edit?usp=sharing&ouid=117357165486381097109&rtpof=true&sd=true', max_count=max_count, count_type=count_type)


 ===========================================: 
Количество токенов в документе : 28165
ЦЕНА запроса: 0.011266  $


In [ ]:
manual_chat_promt = '''Ты престорелый маразматик, специалист по защите информации

У тебя есть требования по защите информации критический информационной инфраструатуры России.
Тебе задает вопрос разработчик информационной системы.
Проконсультируй его, учитывая требования документа с требованиями.

Отвечай максимально точно и используй только информацию из документов.
Документ с информацией для ответа пользователю: '''

# 6 Проверка работы

In [ ]:
ans=answer_index(
    manual_chat_promt,
    'Нужно ли сертифицировать системы, относящиеся к значимым объектам критической информационной инфраструктуры?',
    manual_index,
    verbose = 1,
    top_similar_documents = top_similar_documents
)


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================встроенные в программное обеспечение и (или) программно-аппаратные средства значимых
объектов (при их наличии).
28. Для обеспечения безопасности значимых объектов критической информационной
инфраструктуры должны применяться средства защиты информации, прошедшие оценку на
соответствие требованиям по безопасности в формах обязательной сертификации, испытаний или
приемки.
Средства защиты информации, прошедшие оценку соответствия в форме обязательной
сертификации, применяются в случаях, установленных законодательством Российской Федерации,
а также в случае принятия решения субъектом критической информационной инфраструктуры.
В иных случаях применяются средства защиты информации, прошедшие оценку
соответствия в форме испытаний или приемки, которые проводятся субъектами критической
информационной инфраструктуры самостоятельно или с привлечением организаций, имеющих в
соответствии с з

In [ ]:
ans=answer_index(
    manual_chat_promt,
    'Нужно ли проводить проверки наналичие недекларированных возможностей программного обеспечения незначимых объектов критической информационной инфраструктуры?',
    manual_index,
    verbose = 1,
    top_similar_documents = top_similar_documents
)


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================информации, необходимо учитывать наличие ограничений на возможность их применения
субъектом критической информационной инфраструктуры на любом из принадлежащих ему
значимых объектов критической информационной инфраструктуры со стороны разработчиков
(производителей) или иных лиц.
В значимом объекте не допускаются:
наличие удаленного доступа непосредственно (напрямую) к программным и программно-
аппаратным средствам, в том числе средствам защиты информации, для обновления или
управления со стороны лиц, не являющихся работниками субъекта критической информационной
инфраструктуры;
наличие локального бесконтрольного доступа к программным и программно-аппаратным
средствам, в том числе средствам защиты информации, для обновления или управления со
стороны лиц, не являющихся работниками субъекта критической информационной
инфраструктуры;
передача информации, в том числе технологической 